In [ ]:
%%sh
pygmentize sklearn-boston-housing-generic.py

In [ ]:
%%sh
pygmentize Dockerfile

In [ ]:
%%sh
export REGION=eu-west-1
aws ecr create-repository --repository-name sklearn-custom --region $REGION

In [ ]:
%%sh
# This cell will not run on SageMaker Studio
# The simplest option is to run these commands on your local machine
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t sklearn-custom:estimator -f Dockerfile .
export IMAGE_ID=`docker images -q sklearn-custom:estimator`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/sklearn-custom:estimator

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

print(sagemaker.__version__)

sess = sagemaker.Session()
account_id = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

bucket = sess.default_bucket()                     
prefix = 'sklearn-boston-housing'

training = sess.upload_data(path='../housing.csv', key_prefix=prefix + "/training")
output   = 's3://{}/{}/output/'.format(bucket,prefix)

role = sagemaker.get_execution_role()

sk = Estimator(
             image_uri=account_id+'.dkr.ecr.'+region+'.amazonaws.com/sklearn-custom:estimator',
             role=role,
             instance_count=1, 
             instance_type='ml.m5.large',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1,
                  'random-state': 123
             }
)

sk.fit({'training':training})

In [ ]:
sk_predictor = sk.deploy(instance_type='ml.t2.medium',
                        initial_instance_count=1)

In [ ]:
test_samples = ['0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,4.98',
                '0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,9.14']

sk_predictor.serializer = sagemaker.serializers.CSVSerializer()

response = sk_predictor.predict(test_samples)
print(response)

In [ ]:
sk_predictor.delete_endpoint()